my regions are specified by the Califonria Fire Science Consortium https://www.cafiresci.org/northern-california

metadata on noaa stations https://www.ncei.noaa.gov/access/homr/reports/mshr

noaa dataset api use specified https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation

explanation of dataset lables https://www.ncei.noaa.gov/pub/data/metadata/documents/GSOYReadme.txt



this uses NOAA metadata on station locations to identify stations in area and request those stations from the NOAA API

In [2]:
import requests
import pandas as pd
import io
from shapely.geometry import Point, Polygon
import re

#Fire regions established the california fire science consortium 

region_a = Polygon([(40.467845490773,-121.22314453125),(36.08906040282,-115.7080078125),(34.836349990764,-118.5205078125),(38.980762765016,-121.88232421875)])
region_b = Polygon([(38.632350249008,-121.71203613281),(37.508013519657,-123.42590332031),(33.062115144155,-120.13000488281),(32.06212626392,-116.24084472656),(32.80393039071,-115.95520019531),(34.432317516745,-116.61437988281),(34.739838698614,-118.13049316406),(34.757892712137,-118.56994628906)])
region_c = Polygon([(36.012015045348,-115.47180175781),(34.794192467556,-118.43811035156),(34.39626834484,-116.15295410156),(32.080955638786,-115.66955566406),(32.619260868167,-111.62658691406),(36.047554116355,-115.18615722656)])
region_d = Polygon([(38.692817813434,-121.71752929688),(37.621647607458,-123.36547851562),(39.511264410098,-124.90356445312),(42.251715998571,-124.94750976562),(42.316738909445,-119.36645507812),(38.949602728462,-119.25659179688),(40.604379386497,-121.25610351562),(38.881217307773,-122.02514648438)])
regions = [region_a,region_b, region_c, region_d]



#establish all stations within each region by checking for coordinates inside each region 
stations_a = []
stations_b = []
stations_c = []
stations_d = []

file = open('/home/devin/winternship2023/noaa_climate_data/emshr_lite (5).txt', 'r')
count = 0
while True:
  line = file.readline()
  if not line: #check for end of file 
    break
  
  count += 1
  if count < 3: #insure that only the lines with data are scraped (only past the third index)
    continue
  
  station_id = line[74:85]
  station_coordinate_lat = line[272:281].strip()
  station_coordinate_lon = line[282:292].strip()

  if station_coordinate_lat == '' or station_coordinate_lon == '' or station_id.strip() == '':
    continue
  station_coordinate_lat = float(station_coordinate_lat)
  station_coordinate_lon = float(station_coordinate_lon)
  
  station_coordinate = (station_coordinate_lat, station_coordinate_lon)
  station_coordinate = Point(station_coordinate)
  
  for regional,stational in [(region_a, stations_a), (region_b, stations_b), (region_c, stations_c), (region_d, stations_d)]:
    if regional.contains(station_coordinate):  
      stational.append([station_id, station_coordinate])
  
file.close()



def lat_lon_to_shapely(lat,lon): #this allows us to use basic lat and lon to create a shapely object for initial function
  return Point(lat,lon)

#takes the fire location and returns region of fire 
def find_region(fire_location):
  for region in regions:
    if region.contains(fire_location.centroid):
      return region
    # else:
    #   return None
  




#takes the region and returns a list of stations and their coordinates from metadata 
def region_station_list(region): 
  for regions, stations in [(region_a, stations_a),(region_b, stations_b),(region_c, stations_c),(region_d, stations_d)]:
    if regions == region:
      return stations
 
  



#takes the list of stations and fire location to return stations within a degree of distance from fire
def closest_to_fire_center(station_list, fire_location):
  fire_center = fire_location.centroid
  closest_station_list = []
  for station in station_list:
    if station[1].distance(fire_center) < 1:
      closest_station_list.append((station[0], station[1].distance(fire_center)))
  closest_station_list.sort(key=lambda closest_station_list: closest_station_list[1])
  closest_station_list = closest_station_list[:50]
  return [x[0] for x in closest_station_list]



#takes a string of closest stations and requests their data from noaa api
def request_station_data(closest_stations):
  closest_stations = ','.join(closest_stations)
  
  header = {'token': 'SXCObXMuhoovZxrVZpurQqmsHdvJWyUJ'}
  r=requests.get(f'https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&stations={closest_stations}&&startDate=1950-01-01T00:00:00z&&endDate=2022-01-01&format=csv&bbox=-124.40959,32.534156,-114.131211,42.009518', headers=header)
  if r.status_code != 200:
    print(r.status_code)
    print(r.text)
  df = pd.read_csv(io.StringIO(r.text))
  return df


#takes the station data returns a dataframe row of the averaged values for year of fire 
#add a recurrance here to request more than 50 please 
def filter_station_data(station_df, fire_year):

  station_df = station_df[['DATE','DP01','DP10','DP1X','DSNW','DX32','DX70','DX90','EMSN','EMXP','EMXT','PRCP', 'SNOW','TMAX',]]
  
  station_df = station_df.rename(columns={"DP01": "PDepth>0.01in", "DP10": "PDepth>0.1in", "DP1X": "PDepth>1in", "DSNW": "SDepth>1in", "DX32": "DaysT<32d", "DX70": "DaysT>70d", "DX90": "DaysT>90d",
  "EMSN": "HighestDaySnow", "EMXP": "HighestDayRain", "EMXT": "HighestDayTemp", "PRCP": "TotalRain", "SNOW": "TotalSnow", "TMAX": "AvgMonthMax"})
 
  
  station_df = station_df[station_df['DATE'] == fire_year]
  station_df = station_df.rename(columns = {'DATE': 'Fire_Year'})
  #the method of filling missing values should be changed for final library touches 
  station_df = pd.DataFrame(station_df.mean()).T
  return station_df




event_climate_data was originally meant to be the function to use on dataset but I added convert_event_data() to handle latitude and longitude 

In [ ]:
def event_climate_data(fire_location, fire_year): #function that returns the average local climate data for year of the fire as a row to add to dataset 
  region = find_region(fire_location)
  if region == None:
    return None
  stations = region_station_list(region)
  closest_stations = closest_to_fire_center(stations, fire_location)
  stations_df = request_station_data(closest_stations)
  filtered_df = filter_station_data(stations_df,fire_year)
  return filtered_df
  

The ultimate applicaton of this is to input a fire's lat, lon and year to create a row of the average climate data from the 50 closest stations within a degree of distance from the fire for the year 


In [ ]:
def convert_event_data(lat,lon,fire_year): #this allows for direct conversion from what we have in our fire dataset 
    fire_location = lat_lon_to_shapely(lat, lon)
    return event_climate_data(fire_location, fire_year)

piece by piece function tests:
- if test location is region center it should be in that region 
- region_station_list should return list of IDs and coordinates 
- closest to fire center should be less than total stations and only be IDs
- request station data should return a a full dataframe

In [ ]:
for region in regions: #region function is accurate 
    test_location  = region.centroid
    if find_region(test_location) != region:
        print(f"test failed: {test_location} should have been {region}")

#---------------------------------------------------------------------------------

region = region_a
test_stations = region_station_list(region_a) #this returns a list of stations and coordinates 

#---------------------------------------------------------------------------------

test_closest = closest_to_fire_center(test_stations, region_a.centroid)
print(test_closest)
print(str(len(test_stations)) + ' in region vs ' + str(len(test_closest)) + ' closest') #closest should contain less than test stations

#---------------------------------------------------------------------------------

test_df = request_station_data(test_closest) #returns dataframe of data for stations since 1950
test_df

#---------------------------------------------------------------------------------

filter_station_data(test_df, 2001) #returns a row with the average data for the year 

Now we can use the convert_event_data() function on each row in our fire outcome dataset

convert_event_data(latitude, longitude, year)

- latitude is latitude 
- longitude is latitude 
- year is the year of the event 
- returns a row of the average climate data of the closest 50 noaa stations to be added onto event datasets 


In [ ]:
fire_df = pd.read_csv('/home/devin/winternship2023/datasets/cal1.2.csv')
# fire_df["Fire_Year"] = fire_df["Fire_Year"].apply(lambda x : pd.to_datetime(x).year)
# display(fire_df.head())
#add on the column names we going to add to fire dataset 
new_columns = ['PDepth>0.01in','PDepth>0.1in','PDepth>1in','SDepth>1in','DaysT<32d','DaysT>70d','DaysT>90d','HighestDaySnow','HighestDayRain','HighestDayTemp','TotalRain','TotalSnow','AvgMonthMax']
old_columns = ['Fire_Name','Lat','Long','Fire_Year','Fire_Month','Total_area_burned(ha)','Unchanged(ha)','Low_Severity(ha)','Moderate_Severity(ha)','High_Severity(ha)','FuelTreatment','TSLF(months)']
all_columns = old_columns + new_columns
fire_df[new_columns] = ''
fire_df.head()

Testing our process on a row of dataframe

In [ ]:
index = 0
new_frame = pd.DataFrame(columns = all_columns)

og_row = fire_df.iloc[[index]]
display(og_row)

add_on_row = convert_event_data(fire_df['Lat'][index], fire_df['Long'][index], fire_df['Fire_Year'][index])[new_columns]
display(add_on_row) 

og_row.loc[index, new_columns] = add_on_row.loc[0, new_columns]
display(og_row)

new_frame.loc[index] = og_row.loc[index]
display(new_frame)

In [ ]:
new_frame = pd.DataFrame(columns = all_columns) #where we will stack the new rows 
for index in range(len(fire_df)):
  if find_region(lat_lon_to_shapely(fire_df['Lat'][index], fire_df['Long'][index])) == None: #if the fire is outside of our regions then we skip it
    continue

  og_row = fire_df.iloc[[index]]
  add_on_row = convert_event_data(fire_df['Lat'][index], fire_df['Long'][index], fire_df['Fire_Year'][index])[new_columns]
  
  og_row.loc[index, new_columns] = add_on_row.loc[0, new_columns]
  
  new_frame.loc[index] = og_row.loc[index]


In [ ]:



num_columns = [ 'Lat', 'Long', 'Fire_Year','Fire_Month', 'Total_area_burned(ha)','Moderate_Severity(ha)','High_Severity(ha)','PDepth>0.01in','PDepth>0.1in','PDepth>1in','SDepth>1in','DaysT<32d','DaysT>70d','DaysT>90d','HighestDaySnow','HighestDayRain','HighestDayTemp','TotalRain','TotalSnow','AvgMonthMax']


#replace this method of filling values please god future me I worked on documentation 
for column in num_columns:
  new_frame[column].fillna(value= new_frame[column].mean(), inplace=True)

new_frame.head(20)


In [ ]:
new_frame.to_csv('../datasets/fire_climate_set.csv')


In [ ]:
df = pd.read_csv('../datasets/fire_climate_set.csv')
display(df)

